# Checkpoint 1 — Task 1.1 (Python): Insurance Claims Analysis
**Group:** _A1_  
**Members:** Rishabh Johri, Fayad Haseeb, Harsh Soni, Prashant Tandon 




## Objective
- Load three Excel sheets (Policy, Car, Claim) and create a unified dataset.
- Clean: trim/cast; extract numerics from torque/power; handle duplicates & missing values.
- Explore: class balance, policy feature relationships, categorical summaries.
- Answer required & added questions (Q1–Q6) with Python.
- Quick model-based feature importance for claim drivers.
- Visuals: correlation heatmap & claim pie chart.
- Export cleaned CSV snapshot for SQL/BI.


**Setup**

In [ ]:

import os, warnings, math, numpy as np, pandas as pd
warnings.filterwarnings("ignore")


import matplotlib.pyplot as plt


In [ ]:

POLICY_XLSX = "/mnt/data/Policy features.xlsx"
CAR_XLSX    = "/mnt/data/Car features.xlsx"
CLAIM_XLSX  = "/mnt/data/Insurance claim.xlsx"

for p in [POLICY_XLSX, CAR_XLSX, CLAIM_XLSX]:
    assert os.path.exists(p), f"Missing file: {p}"
print("All input files found.")

## 1) Load & Merge

In [ ]:
policy = pd.read_excel(POLICY_XLSX)
car    = pd.read_excel(CAR_XLSX)
claim  = pd.read_excel(CLAIM_XLSX)

df = policy.merge(car, on="policy_id").merge(claim, on="policy_id")
print("Shapes:", policy.shape, car.shape, claim.shape, "→ merged:", df.shape)
display(df.head(5))

## 2) Cleaning & Casting
- Standardize column names (lower_snake).
- Trim strings; remove exact duplicates.
- Extract numerics from `max_torque` / `max_power`.
- Cast boolean flags & target to {0,1}.
- Quick missingness report.

In [ ]:

df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]


for c in df.select_dtypes(include=["object"]).columns:
    df[c] = df[c].astype(str).str.strip()


dup = df.duplicated().sum()
df = df.drop_duplicates()
print(f"Removed duplicates: {dup} (now {df.shape})")


def extract_num(s):
    return pd.to_numeric(pd.Series(s, dtype="object").astype(str).str.extract(r'([-+]?\d*\.?\d+)')[0], errors="coerce")

if "max_torque" in df.columns:
    df["max_torque_num"] = extract_num(df["max_torque"])
if "max_power" in df.columns:
    df["max_power_num"] = extract_num(df["max_power"])


for c in [col for col in df.columns if col.startswith("is_")] + ["is_claim"]:
    if c in df.columns:
        df[c] = (df[c].map({"Yes":1,"No":0,"True":1,"False":0,1:1,0:0}).fillna(df[c]).astype(float))


miss = df.isna().mean().sort_values(ascending=False)
print("Top-20 missingness:")
display(miss.head(20))

## 3) Dataset Snapshot
Rows/Cols, duplicates, and class balance.

In [ ]:
print("Shape:", df.shape)
print("Duplicates:", df.duplicated().sum())
label_counts = df["is_claim"].value_counts(dropna=False)
label_pct = (label_counts/label_counts.sum()*100).round(2)
display(pd.DataFrame({"count": label_counts, "pct": label_pct}))

## 4) Analyses — Required Questions

### Q1. Analyse policy features. Any relation with `is_claim`?
Compute Pearson correlations for policy-related numerics.


In [ ]:

policy_cols = [c for c in ["policy_tenure","age_of_car","age_of_policyholder","population_density","is_claim"] if c in df.columns]
corr_policy = df[policy_cols].corr(numeric_only=True)["is_claim"].sort_values(ascending=False)
display(corr_policy.to_frame("pearson_corr"))


### Q2. Is the dataset balanced? Are there duplicates?
Already summarized in the snapshot table above.

### Q3. Investigate car features and find important features affecting `is_claim`.
Use a quick model-based importance (Random Forest) on preprocessed data (OHE + imputation).

In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

y = df["is_claim"].fillna(0).astype(int)
X = df.drop(columns=["is_claim","policy_id"], errors="ignore")

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object"]).columns.tolist()

pre = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse=False, min_frequency=0.01), cat_cols),
    ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols)
])

rf = RandomForestClassifier(n_estimators=200, class_weight="balanced", n_jobs=-1, random_state=42)

pipe = Pipeline([("prep", pre), ("rf", rf)])

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
pipe.fit(X_tr, y_tr)
proba = pipe.predict_proba(X_te)[:,1]
auc = roc_auc_score(y_te, proba)
print("Holdout ROC-AUC:", round(auc, 4))


ohe = pipe.named_steps["prep"].named_transformers_["cat"]
cat_feats = list(ohe.get_feature_names_out(cat_cols)) if cat_cols else []
all_feats = cat_feats + num_cols
imp = pd.DataFrame({"feature": all_feats[:len(pipe.named_steps['rf'].feature_importances_)],
                    "importance": pipe.named_steps["rf"].feature_importances_}).sort_values("importance", ascending=False)
display(imp.head(25))

## 5) Analyses — Additional Questions

### Q4. Which **car segment** has the highest claim percentage?
### Q5. Do **safety features** (airbags, ESC, NCAP) reduce claims?
### Q6. Do **fuel type** and **transmission** influence claims?

In [ ]:

if "segment" in df.columns:
    seg_rate = df.groupby("segment")["is_claim"].mean().sort_values(ascending=False)
    display(seg_rate.to_frame("claim_rate"))
else:
    print("segment column not found.")


to_check = []
if "ncap_rating" in df.columns: to_check.append("ncap_rating")
if "airbags" in df.columns: to_check.append("airbags")
if "is_esc" in df.columns: to_check.append("is_esc")

for col in to_check:
    grp = df.groupby(col)["is_claim"].mean().sort_values(ascending=False)
    print(f"\nClaim rate by {col}:")
    display(grp.to_frame("claim_rate"))


for col in ["fuel_type","transmission_type"]:
    if col in df.columns:
        grp = df.groupby(col)["is_claim"].mean().sort_values(ascending=False)
        print(f"\nClaim rate by {col}:")
        display(grp.to_frame("claim_rate"))

## 6) Visuals (Matplotlib only)
- **Correlation Heatmap** (numeric features + `is_claim`).
- **Pie Chart** for `is_claim` distribution.


In [ ]:
# Correlation heatmap (matplotlib only)
num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
if "is_claim" in num_cols_all:
    cols_for_corr = [c for c in num_cols_all if c != "policy_id"]
    C = df[cols_for_corr].corr(numeric_only=True).values
    labels = cols_for_corr

    plt.figure(figsize=(10,8))
    im = plt.imshow(C, aspect='auto')
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks(ticks=range(len(labels)), labels=labels, rotation=90)
    plt.yticks(ticks=range(len(labels)), labels=labels)
    plt.tight_layout()
    plt.show()

# Pie chart for class balance
counts = df["is_claim"].value_counts().sort_index()
plt.figure(figsize=(6,6))
plt.pie(counts, labels=[str(i) for i in counts.index], autopct="%1.1f%%", startangle=90)
plt.title("Insurance Claim Distribution (is_claim)")
plt.show()

## 7) Export Cleaned Snapshot
Produces a CSV to reuse in SQL (Checkpoint 1.2) and Power BI (Checkpoint 2.1).

In [ ]:
out_csv = "/mnt/data/cleaned_insurance.csv"
df.to_csv(out_csv, index=False)
print("Saved:", out_csv)

## 8) Conclusions 
- Dataset is **highly imbalanced** (≈ 94% no-claim vs 6% claim).
- Policy features (e.g., `policy_tenure`) show stronger association with `is_claim` than many raw car specs.
- Safety features (higher `ncap_rating`, `is_esc`=1, more `airbags`) trend toward **lower** claim rates.
- Some segments and fuel/transmission types exhibit higher claim propensity; capture these in targeted strategies.
- Use class-weighting/stratification for downstream ML; prefer ROC-AUC/PR-AUC for evaluation.
